In [1]:
import sys
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("spark://spark-master:7077").appName("SparkByIESB").getOrCreate()

22/06/09 23:09:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#df = spark.read.option("header","true").option("inferSchema","true").csv("cesta.csv")
df = spark.read.option("header","true").csv("cesta.csv")#.drop("student_id")
#print(df)
#df.printSchema()

from pyspark.sql.functions import collect_list, col
cestas = df.rdd.map(lambda x: (x[0], ["assignment_update"+x[1],"assignment_upload"+x[2],"assignment_view"+x[3]]))
df2 = cestas.toDF(["id","items"])
df2.show()

#rdd = df.rdd.map(lambda x: (x[0],x[1:]))
#rdd.printSchema()

#from pyspark.sql import functions as F
#df2 = df.groupBy("student_id").agg(F.collect_list("student_id")).sort('student_id')
#df2.printSchema()
#df2.show()

+---+--------------------+
| id|               items|
+---+--------------------+
|319|[assignment_updat...|
|320|[assignment_updat...|
|321|[assignment_updat...|
|322|[assignment_updat...|
|323|[assignment_updat...|
|324|[assignment_updat...|
|325|[assignment_updat...|
|326|[assignment_updat...|
|327|[assignment_updat...|
|328|[assignment_updat...|
|329|[assignment_updat...|
|330|[assignment_updat...|
|331|[assignment_updat...|
|332|[assignment_updat...|
|333|[assignment_updat...|
|334|[assignment_updat...|
|335|[assignment_updat...|
|336|[assignment_updat...|
|337|[assignment_updat...|
|338|[assignment_updat...|
+---+--------------------+
only showing top 20 rows



In [3]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(df2)

In [4]:
# Display frequent itemsets.
model.freqItemsets.show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|  [assignment_view3]| 7353|
|[assignment_view3...| 7353|
|[assignment_upload4]| 8646|
|[assignment_uploa...| 8646|
|[assignment_update4]|14459|
+--------------------+-----+



In [5]:
# Display generated association rules.
model.associationRules.show()

+--------------------+--------------------+----------+----+
|          antecedent|          consequent|confidence|lift|
+--------------------+--------------------+----------+----+
|[assignment_upload4]|[assignment_update4]|       1.0| 1.0|
|  [assignment_view3]|[assignment_update4]|       1.0| 1.0|
+--------------------+--------------------+----------+----+

